<a href="https://colab.research.google.com/github/Ryo0jp/Udemy_Website_RAG/blob/main/Udemy_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##加工済み情報に対するRAG

In [ ]:
question = "2023年の第1事業部の売上はどのくらい？"

documents = [
    "2023年上期売上200億円、下期売上300億円",
    "2023年第1事業部売上300億円、第2事業部売上150億円、第3事業部売上50億円",
    "2024年は全社で1000億円の売上を目指す"
]



In [1]:
!pip install httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [4]:
from openai import OpenAI
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import userdata


os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
client = OpenAI()

text =  "2023年上期売上200億円、下期売上300億円"

def vectorize_text(text):
  response = client.embeddings.create(
      input = text,
      model = "text-embedding-3-small"
  )
  return response.data[0].embedding

question = "2023年の第1事業部の売上はどのくらい？"

documents = [
    "2023年上期売上200億円、下期売上300億円",
    "2023年第1事業部売上300億円、第2事業部売上150億円、第3事業部売上50億円",
    "2024年は全社で1000億円の売上を目指す"
]


vectors = [vectorize_text(doc) for doc in documents]
question_vector = vectorize_text(question)



In [9]:
max_similarity = 0
most_similar_index = 0

for index, vector in enumerate(vectors):
  similarity = cosine_similarity([question_vector], [vector])[0][0]
  print(documents[index], ":", similarity)
  if similarity > max_similarity:
    max_similarity = similarity
    most_similar_index = index

print(documents[most_similar_index])

2023年上期売上200億円、下期売上300億円 : 0.6032776608312375
2023年第1事業部売上300億円、第2事業部売上150億円、第3事業部売上50億円 : 0.8517831817092729
2024年は全社で1000億円の売上を目指す : 0.6197631928209378
2023年第1事業部売上300億円、第2事業部売上150億円、第3事業部売上50億円


In [13]:
prompt = f'''以下の質問に以下の情報をベースにして答えてください。
【ユーザーの質問】
{question}

【情報】
{documents[most_similar_index]}
'''

response = client.completions.create(
    model = "gpt-3.5-turbo-instruct",
    prompt = prompt,
    max_tokens = 200
)

print(response.choices[0].text)


2023年の第1事業部の売上は300億円です。


##特定のURLの情報にRAGを使ってアクセスしてチャットbotから回答を得る

In [25]:
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity

url = "https://toukei-lab.com/achademy/?page_id=1619"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

t_all = []
text_nodes = soup.find_all("div")
#len(text_nodes)
for t in text_nodes:
  t_all.append(t.text.replace("\t" , "").replace("\n" , ""))

joined_text = "".join(t.text.replace("\t" , "").replace("\n" , "") for t in text_nodes)

## 1文で書かない場合 ##
# t_all = []
# for t in text_nodes:
#   t_all.append(t.text.replace("\t", "").replace("\n", ""))

# joined_text = "".join(t_all)

In [26]:
joined_text

'MENUTOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログインお問い合わせLINEで無料相談する法人のお客様はこちら定期購読の解約・中断方法 AIデータサイエンス特化スクール TOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログインお問い合わせLINEで無料相談する法人のお客様はこちら定期購読の解約・中断方法 TOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログインお問い合わせLINEで無料相談する法人のお客様はこちら定期購読の解約・中断方法プラン一覧 プラン一覧ライトプランライトプラン1280 円/ 月全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（4回分）専用コミュニティへの参加試してみるライトプランでは当サービス「スタアカ」の全コースが全て受講し放題です。※1コースごとの価格ではなくライトプランに登録いただくと全コースが受講可能になりますデジタルトランスフォーメーション(DX)概要コースデータサイエンティスト概要コースPython基礎コースデータ加工集計可視化コース統計学概要コース機械学習概要コース機械学習実装コースディープラーニングコースSQL基礎コースなどなど(※1ヶ月ごとに新しいカリキュラム追加)※決済いただいた日付から1ヶ月間のご受講が始まります。次回の請求はご受講開始から1ヶ月後です。プレミアムプランプレミアムプラン\\ただいまセール中/149,800 円129,800 円/ 買い切り全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（4回分）専用コミュニティへの参加試してみるプレミアムプランは確実に実力を付けるためのプランです。・スタアカ専用QAフォームでの24時間以内の返信対応（購入後ずっと）・1回30分の当サービス運営代表（twitter.com/statistics1012）含む現役データサイエンティストとのマンツーマンメンタリング（4回分）・全コンテンツの受講（ずっと）・運営代表（twitter.com/statistics

In [28]:
len(joined_text)

11109

In [29]:
chunk_size = 400
overlap = 50
chunks = []
start = 0

while start + chunk_size <= len(joined_text):
  chunks.append(joined_text[start:start + chunk_size])
  start += (chunk_size - overlap)

if start < len(joined_text):
  chunks.append(joined_text[-chunk_size:])

In [30]:
for i, chunk in enumerate(chunks):
  print(f"chunk{i+1} : {chunk[:50]}...{chunk[-50:]}")

chunk1 : MENUTOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミアムプランを...断方法プラン一覧 プラン一覧ライトプランライトプラン1280 円/ 月全コース受講し放題フォーラム質
chunk2 : 断方法プラン一覧 プラン一覧ライトプランライトプラン1280 円/ 月全コース受講し放題フォーラム質...月後です。プレミアムプランプレミアムプラン\ただいまセール中/149,800 円129,800 円/
chunk3 : 月後です。プレミアムプランプレミアムプラン\ただいまセール中/149,800 円129,800 円/...ンについて詳しく見るオーダーメイドプランオーダーメイドプラン398,000 円 全コース受講し放題フ
chunk4 : ンについて詳しく見るオーダーメイドプランオーダーメイドプラン398,000 円 全コース受講し放題フ...myコース25個以上の無料贈呈・プレミアムプラン専用のコミュニティに参加オーダーメイドプランについて
chunk5 : myコース25個以上の無料贈呈・プレミアムプラン専用のコミュニティに参加オーダーメイドプランについて...プスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログ
chunk6 : プスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログ...はこちら定期購読の解約・中断方法  TOPコースコース一覧コースロードマップスタアカ チュートリアル
chunk7 : はこちら定期購読の解約・中断方法  TOPコースコース一覧コースロードマップスタアカ チュートリアル...プラン一覧 プラン一覧ライトプランライトプラン1280 円/ 月全コース受講し放題フォーラム質問し放
chunk8 : プラン一覧 プラン一覧ライトプランライトプラン1280 円/ 月全コース受講し放題フォーラム質問し放...す。プレミアムプランプレミアムプラン\ただいまセール中/149,800 円129,800 円/ 買い
chunk9 : す。プレミアムプランプレミアムプラン\ただいまセール中/149,800 円129,800 円/ 買い...いて詳しく見るオーダーメイドプランオーダーメイドプラン398,000

In [3]:
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import userdata


os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
client = OpenAI()


def scrape_article(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser")
  text_nodes = soup.find_all("div")
  joined_text = "".join(t.text.replace("\t", "").replace("\n", "") for t in text_nodes)

  return joined_text


def chunk_text(text, chunk_size, overlap):
  chunks = []
  start = 0
  while start + chunk_size <= len(text):
    chunks.append(text[start:start + chunk_size])
    start += (chunk_size - overlap)
  if start < len(text):
    chunks.append(text[-chunk_size:])

  return chunks

def vectorize_text(text):
  response = client.embeddings.create(
      input = text,
      model = "text-embedding-3-small"
  )
  return response.data[0].embedding


def find_most_similar(question_vector, vectors, documents):
  similarities = []

  for index, vector in enumerate(vectors):
    similarity = cosine_similarity([question_vector], [vector])[0][0]
    similarities.append([similarity, index])

  similarities.sort(reverse=True, key=lambda x: x[0])
  top_documents = [documents[index] for similarity, index in similarities[:2]]

  return top_documents


def ask_question(question, context):
  prompt = f'''以下の質問に以下の情報をベースにして答えて下さい。
  [ユーザーの質問]
  {question}

  [情報]
  {context}
  '''
  print(prompt)
  response = client.completions.create(
      model="gpt-3.5-turbo-instruct",
      prompt=prompt,
      max_tokens=200
  )

  return response.choices[0].text


url = "https://toukei-lab.com/achademy/?page_id=1619"
chunk_size = 400
overlap = 50

article_text = scrape_article(url)
text_chunks = chunk_text(article_text, chunk_size, overlap)

vectors = [vectorize_text(doc) for doc in text_chunks]

question = "オーダーメイドプランの価格はいくらですか？"
question_vector = vectorize_text(question)

similar_document = find_most_similar(question_vector, vectors, text_chunks)

answer = ask_question(question, similar_document)
print(answer)

以下の質問に以下の情報をベースにして答えて下さい。
  [ユーザーの質問]
  オーダーメイドプランの価格はいくらですか？

  [情報]
  ['いて詳しく見るオーダーメイドプランオーダーメイドプラン398,000 円 全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（8回分）専用コミュニティへの参加オーダーメイドの学習試してみるオーダーメイドプランは一人一人の目指す姿を実現できるように徹底的に伴走させていただくプランです・スタアカ専用QAフォームでの24時間以内の返信対応（購入後ずっと）・1回30分の当サービス運営代表（twitter.com/statistics1012）とのマンツーマンメンタリング（月1回×8ヶ月）・前半4ヶ月でスタアカカリキュラムを終わらせ残りの4ヶ月でオーダーメイドの学習・全コンテンツの受講（ずっと）・運営代表（twitter.com/statistics1012）のUdemyコース25個以上の無料贈呈・プレミアムプラン専用のコミュニティに参加オーダーメイドプランについて詳しく', 'ンについて詳しく見るオーダーメイドプランオーダーメイドプラン398,000 円 全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（8回分）専用コミュニティへの参加オーダーメイドの学習試してみるオーダーメイドプランは一人一人の目指す姿を実現できるように徹底的に伴走させていただくプランです・スタアカ専用QAフォームでの24時間以内の返信対応（購入後ずっと）・1回30分の当サービス運営代表（twitter.com/statistics1012）とのマンツーマンメンタリング（月1回×8ヶ月）・前半4ヶ月でスタアカカリキュラムを終わらせ残りの4ヶ月でオーダーメイドの学習・全コンテンツの受講（ずっと）・運営代表（twitter.com/statistics1012）のUdemyコース25個以上の無料贈呈・プレミアムプラン専用のコミュニティに参加オーダーメイドプランについて']
  

オーダーメイドプランの価格は398,000円です。詳細はお問い合わせフォームやプランページをご確認いただけますと幸いです。
